# Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# creo el cvs a tratar

In [2]:
import pandas as pd
malicious_data = pd.read_csv("maligno.csv")
non_malicious_data = pd.read_csv("good.csv")

# Limpieza de características

In [3]:
#Elimina datos
columnas_a_eliminar = ['exaddr','engine_type','engine_id','dst_mask','src_mask', 'dst_mask','src_as','dst_as','#:unix_secs','unix_nsecs','sysuptime','first','last','nexthop']
malicious_data = malicious_data.drop(columnas_a_eliminar, axis=1)
non_malicious_data = non_malicious_data.drop(columnas_a_eliminar, axis=1)

In [4]:
import socket

# Función para convertir direcciones IP en valores enteros
def convert_ip_to_int(ip):
    try:
        return int.from_bytes(socket.inet_aton(ip), byteorder='big')
    except OSError:
        return ip  # Retorna la dirección IP no válida sin convertir




In [5]:

# Aplicar la función lambda a la columna srcaddr y almacenar los resultados en una nueva columna
malicious_data['srcaddr'] = malicious_data['srcaddr'].apply(lambda x: int.from_bytes(socket.inet_aton(x), byteorder='big'))
malicious_data['dstaddr'] = malicious_data['dstaddr'].apply(lambda x: int.from_bytes(socket.inet_aton(x), byteorder='big'))

# Eliminar las filas que contienen valores no válidos
#malicious_data = malicious_data.dropna(subset=['srcaddr'])
#malicious_data = malicious_data.dropna(subset=['dstaddr'])

malicious_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176447 entries, 0 to 176446
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   dpkts      176447 non-null  int64
 1   doctets    176447 non-null  int64
 2   srcaddr    176447 non-null  int64
 3   dstaddr    176447 non-null  int64
 4   input      176447 non-null  int64
 5   output     176447 non-null  int64
 6   srcport    176447 non-null  int64
 7   dstport    176447 non-null  int64
 8   prot       176447 non-null  int64
 9   tos        176447 non-null  int64
 10  tcp_flags  176447 non-null  int64
dtypes: int64(11)
memory usage: 14.8 MB


In [6]:
# Aplicar la función a la columna 'srcaddr'
non_malicious_data['srcaddr'] = non_malicious_data['srcaddr'].apply(convert_ip_to_int)
non_malicious_data['dstaddr'] = non_malicious_data['dstaddr'].apply(convert_ip_to_int)

# Eliminar las filas que contienen valores no válidos
#non_malicious_data = non_malicious_data.dropna(subset=['srcaddr'])
#non_malicious_data = non_malicious_data.dropna(subset=['dstaddr'])

malicious_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176447 entries, 0 to 176446
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   dpkts      176447 non-null  int64
 1   doctets    176447 non-null  int64
 2   srcaddr    176447 non-null  int64
 3   dstaddr    176447 non-null  int64
 4   input      176447 non-null  int64
 5   output     176447 non-null  int64
 6   srcport    176447 non-null  int64
 7   dstport    176447 non-null  int64
 8   prot       176447 non-null  int64
 9   tos        176447 non-null  int64
 10  tcp_flags  176447 non-null  int64
dtypes: int64(11)
memory usage: 14.8 MB


# Normaliza


In [8]:
from sklearn.preprocessing import MinMaxScaler

# Normalizar los datos en el dataframe malicious_data
# La función fit_transform() ajusta los datos y los transforma al rango especificado
# En este caso, se ajustan los datos para que estén en un rango entre 0 y 1
datos_normalizados_malicius = MinMaxScaler().fit_transform(malicious_data)

# Normalizar los datos en el dataframe non_malicious_data
# La función fit_transform() ajusta los datos y los transforma al rango especificado
# En este caso, se ajustan los datos para que estén en un rango entre 0 y 1
datos_normalizados_non_malicius = MinMaxScaler().fit_transform(non_malicious_data)


# Clasificación


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [10]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba

X = pd.concat([malicious_data, non_malicious_data]) # Concatenar ambos conjuntos
y = [1]*len(malicious_data) + [0]*len(non_malicious_data) # Etiquetas de clase: 1 para datos maliciosos, 0 para no maliciosos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)


# KNeighbors

In [11]:

model = KNeighborsClassifier()
# Se entrena el modelo con los datos de entrenamiento
model.fit(X_train, y_train)
# Se hacen predicciones con los datos de prueba
y_pred = model.predict(X_test)
# Se calcula la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print("accuracy: " + str(accuracy)+"\n")
print("Matriz de confusión")

# Se muestra la matriz de confusión utilizando la función confusion_matrix() de scikit-learn
# que toma como argumentos los valores verdaderos y las predicciones del modelo

confusion_matrix(y_test, y_pred)

accuracy: 0.999489933266269

Matriz de confusión


array([[35116,     4],
       [   32, 35427]])

# LogisticRegression

In [12]:
model = LogisticRegression()
# Se entrena el modelo con los datos de entrenamiento
model.fit(X_train, y_train)
# Se hacen predicciones con los datos de prueba
y_pred = model.predict(X_test)
# Se calcula la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print("accuracy: " + str(accuracy)+"\n")
print("Matriz de confusión")

# Se muestra la matriz de confusión utilizando la función confusion_matrix() de scikit-learn
# que toma como argumentos los valores verdaderos y las predicciones del modelo

confusion_matrix(y_test, y_pred)

accuracy: 0.4975984357953499

Matriz de confusión


array([[35120,     0],
       [35459,     0]])

# LinearSVC

In [ ]:

model = LinearSVC(max_iter=10000)
# Se entrena el modelo con los datos de entrenamiento
model.fit(X_train, y_train)
# Se hacen predicciones con los datos de prueba
y_pred = model.predict(X_test)
# Se calcula la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print("accuracy: " + str(accuracy)+"\n")
print("Matriz de confusión")

# Se muestra la matriz de confusión utilizando la función confusion_matrix() de scikit-learn
# que toma como argumentos los valores verdaderos y las predicciones del modelo

confusion_matrix(y_test, y_pred)

# Perceptron

In [14]:
model = Perceptron()
# Se entrena el modelo con los datos de entrenamiento
model.fit(X_train, y_train)
# Se hacen predicciones con los datos de prueba
y_pred = model.predict(X_test)
# Se calcula la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print("accuracy: " + str(accuracy)+"\n")
print("Matriz de confusión")

# Se muestra la matriz de confusión utilizando la función confusion_matrix() de scikit-learn
# que toma como argumentos los valores verdaderos y las predicciones del modelo

confusion_matrix(y_test, y_pred)

accuracy: 0.2754360362147381

Matriz de confusión


array([[19440, 15680],
       [35459,     0]])

# RandomForestClassifier

In [15]:
model = RandomForestClassifier()
# Se entrena el modelo con los datos de entrenamiento
model.fit(X_train, y_train)
# Se hacen predicciones con los datos de prueba
y_pred = model.predict(X_test)
# Se calcula la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print("accuracy: " + str(accuracy)+"\n")
print("Matriz de confusión")

# Se muestra la matriz de confusión utilizando la función confusion_matrix() de scikit-learn
# que toma como argumentos los valores verdaderos y las predicciones del modelo

confusion_matrix(y_test, y_pred)

accuracy: 1.0

Matriz de confusión


array([[35120,     0],
       [    0, 35459]])